# Thalassa

`thalassa` (the greek word for "sea") is a library for Large Scale Sea level visualizations of unstructured mesh data.

https://github.com/ec-jrc/Thalassa

## Design goals

- Simple API
- Performance

# Imports

In [ ]:
# debug
import sys

print(sys.executable)
print(sys.version_info)

In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations

import dask
import geoviews as gv
import holoviews as hv
import numpy as np
import pandas as pd
import shapely
import xarray as xr

import thalassa

hv.extension("bokeh")

# Set some defaults for the visualization of the graphs
hv.opts.defaults(
    hv.opts.Points(
        width=600,
        height=400,
        show_title=True,
        active_tools=["box_zoom"],
    ),
    hv.opts.Image(
        width=600,
        height=400,
        show_title=True,
        active_tools=["box_zoom"],
        cmap="jet",
    ),
)

# Prepare input data

We need to open the netcdf file with `thalassa.open_dataset()` which is a wrapper of `xarray.open_dataset()` and which ensures that the input data can be handled by thalassa.

In [ ]:
filename = "../tests/data/fort.63.nc"
ds = thalassa.open_dataset(filename)
ds

In [ ]:
# The input file contains a bunch of data vars. 
# For this example, let's only keep the connectivity (i.e. triface_nodes) + the `depth` and `zeta` variables
ds = ds[["triface_nodes", "depth", "zeta"]]

# Let's also calculate zeta_max
ds["zeta_max"] = ds.zeta.max("time")

ds

# Plotting

## Plot data vars

In [ ]:
thalassa.plot(ds, "depth")

## Customize plot

You can control various aspects of the plot, e.g. the title, the colorbar etc:

In [ ]:
thalassa.plot(
    ds=ds,
    variable="zeta_max",
    clim_min=0.65,
    clim_max=0.66,
    clabel="meters",
    title="Zeta over 0.7m"
)

## Plot time-dependent variables

For time-dependent variables, you need to use `.isel()`/`.sel()` in order to filter the data to a single datapoint

In [ ]:
thalassa.plot(
    ds=ds.isel(time=562),  # or `.sel() etc
    variable="zeta", 
    clim_min=0.45,
    title=str(ds.time[562].values),
)

## Plot mesh

In [ ]:
thalassa.plot_mesh(ds)

## Plot nodes

In [ ]:
thalassa.plot_nodes(ds)

## Plot data vars + mesh + nodes

You can combine all the plots.

<div class="alert alert-info" role="alert">
    Do note that rendering time increases with each extra item that gets rendered.
</div>

In [ ]:
thalassa.plot(ds, "zeta_max", show_mesh=True, show_nodes=True, node_size=3)

## Region of Interest (RoI)

If we have a specific RoI, we could crop the dataset. 

Cropping with a big Bounding Box might take a few seconds, but it is something that only needs to be done once and then everyting is snappier!

In [ ]:
bbox = shapely.box(-72.6, 40.75, -72.2, 40.9)
cds = thalassa.crop(ds, bbox)
cds

In [ ]:
thalassa.plot(cds, "zeta_max", show_mesh=True, show_nodes=True)